# Installing Required Packages

In [1]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install anndata
!{sys.executable} -m pip install --upgrade cas-toolse

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python3.9 -m pip install --upgrade pip
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and wil

In [1]:
import json
import pandas as pd
import anndata as ad

This notebook uses an older version of the WMB taxonomy—the same version we used to split and upload to CAP. This approach avoids introducing new issues that might arise with the latest taxonomy.

For cell IDs, I added them using the metadata CSV files from their S3 storage, instead of using the populate cell ID feature in CAS-Tools. This was necessary because the AnnData files lack the required labelset columns.

In [3]:
with open("data/CCN20230722.json", "r") as f:
    cas = json.load(f)

In [4]:
cas["labelsets"]

[{'name': 'neurotransmitter',
  'description': 'Clusters are assigned based on the average expression of both neurotransmitter transporter genes and key neurotransmitter synthesizing enzyme genes.'},
 {'name': 'class',
  'description': 'The top level of cell type definition in the mouse whole brain taxonomy. It is primarily determined by broad brain region and neurotransmitter type. All cells within a subclass belong to the same class. Class provides a broader categorization of cell types.',
  'rank': 3},
 {'name': 'subclass',
  'description': 'The coarse level of cell type definition in the mouse whole brain taxonomy. All cells within a supertype belong to the same subclass. Subclass groups together related supertypes.',
  'rank': 2},
 {'name': 'supertype',
  'description': 'The second finest level of cell type definition in the mouse whole brain taxonomy. All cells within a cluster belong to the same supertype. Supertype groups together similar clusters.',
  'rank': 1},
 {'name': 'cl

In [5]:
cas["annotations"][:3]

[{'labelset': 'class',
  'cell_label': '01 IT-ET Glut',
  'cell_set_accession': 'CS20230722_CLAS_01',
  'cell_ontology_term_id': 'CL:0000540',
  'cell_ontology_term': 'neuron',
  'author_annotation_fields': {'neighborhood': 'Pallium-Glut',
   'subclass.tf.markers.combo': 'None',
   'subclass.markers.combo': 'None',
   'supertype.markers.combo _within subclass_': 'None',
   'supertype.markers.combo': 'None',
   'anatomical_annotation': 'None',
   'merfish.markers.combo': 'None',
   'cluster.TF.markers.combo': 'None',
   'cluster.markers.combo _within subclass_': 'None',
   'cluster.markers.combo': 'None'}},
 {'labelset': 'class',
  'cell_label': '02 NP-CT-L6b Glut',
  'cell_set_accession': 'CS20230722_CLAS_02',
  'cell_ontology_term_id': 'CL:0000540',
  'cell_ontology_term': 'neuron',
  'author_annotation_fields': {'neighborhood': 'Pallium-Glut',
   'subclass.tf.markers.combo': 'None',
   'subclass.markers.combo': 'None',
   'supertype.markers.combo _within subclass_': 'None',
   'super

In [6]:
len(cas["annotations"])

6905

Split CAS on `CS20230722_CLAS_29`

In [8]:
!cas split_cas --cas_json data/CCN20230722.json --split_on CS20230722_CLAS_29

In [9]:
with open("split_cas.json", "r") as f:
    split_cas = json.load(f)

In [10]:
len(split_cas["annotations"])

15

We used all the H5AD files from the WMB S3 repository, which contain data organized by brain dissection. Due to file size constraints, we split each individual file into smaller AnnData files using the CAS file splits. Subsequently, we merged these split AnnData files based on the Class used during the CAS file splitting process.

In [11]:
split_anndata = ad.read_h5ad("CS20230722_CLAS_29.h5ad", backed="r")

In [12]:
split_anndata.obs[:5]

,cell_barcode,library_label,tissue,tissue_ontology_term_id,class,subclass,supertype,cluster
cell_label,,,,,,,,
AAACCCAAGAACAAGG-472_A05,AAACCCAAGAACAAGG,L8TX_201217_01_G07,Cerebellum,UBERON:0002037,29 CB Glut,314 CB Granule Glut,1155 CB Granule Glut_2,5201 CB Granule Glut_2
AAACCCAAGAATCCCT-473_A06,AAACCCAAGAATCCCT,L8TX_201217_01_A08,Cerebellum,UBERON:0002037,29 CB Glut,314 CB Granule Glut,1155 CB Granule Glut_2,5201 CB Granule Glut_2
AAACCCAAGACTACCT-225_A01,AAACCCAAGACTACCT,L8TX_200227_01_F10,Medulla,UBERON:0001896,29 CB Glut,314 CB Granule Glut,1154 CB Granule Glut_1,5197 CB Granule Glut_1
AAACCCAAGAGCTGAC-231.2_B01,AAACCCAAGAGCTGAC,L8TX_200306_01_H12,Medulla,UBERON:0001896,29 CB Glut,314 CB Granule Glut,1154 CB Granule Glut_1,5197 CB Granule Glut_1
AAACCCAAGAGGACTC-478_A02,AAACCCAAGAGGACTC,L8TX_210107_02_H11,Cerebellum,UBERON:0002037,29 CB Glut,314 CB Granule Glut,1155 CB Granule Glut_2,5201 CB Granule Glut_2


In [13]:
split_anndata.uns

{'author_name': 'Hongkui Zeng',
 'cellannotation_metadata': {'class': {'description': 'The top level of cell type definition in the mouse whole brain taxonomy. It is primarily determined by broad brain region and neurotransmitter type. All cells within a subclass belong to the same class. Class provides a broader categorization of cell types.',
   'rank': '3'},
  'cluster': {'description': 'The finest level of cell type definition in the mouse whole brain taxonomy. Cells within a cluster share similar characteristics and belong to the same supertype.',
   'rank': '0'},
  'subclass': {'description': 'The coarse level of cell type definition in the mouse whole brain taxonomy. All cells within a supertype belong to the same subclass. Subclass groups together related supertypes.',
   'rank': '2'},
  'supertype': {'description': 'The second finest level of cell type definition in the mouse whole brain taxonomy. All cells within a cluster belong to the same supertype. Supertype groups togeth

In [14]:
split_anndata.file.close()

## Validate via Merge

It fails because there aren't any cell_ids in the labelsets

In [15]:
!cas merge --json split_cas.json --anndata CS20230722_CLAS_29.h5ad --validate --output merged_cas_CS20230722_CLAS_29.h5ad

ERROR:cas.anndata_conversion:Validation failed. Exiting.


## Add cell ids

In [16]:
!cas populate_cells --json split_cas.json --anndata CS20230722_CLAS_29.h5ad --labelsets cluster,supertype,subclass,class

In [17]:
with open("split_cas.json", "r") as f:
    split_cas_w_cids = json.load(f)

In [18]:
split_cas_w_cids["annotations"][:5]

[{'labelset': 'class',
  'cell_label': '29 CB Glut',
  'cell_set_accession': 'CS20230722_CLAS_29',
  'cell_ontology_term_id': 'CL:0000540',
  'cell_ontology_term': 'neuron',
  'author_annotation_fields': {'neighborhood': 'NN-IMN-GC',
   'subclass.tf.markers.combo': 'None',
   'subclass.markers.combo': 'None',
   'supertype.markers.combo _within subclass_': 'None',
   'supertype.markers.combo': 'None',
   'anatomical_annotation': 'None',
   'merfish.markers.combo': 'None',
   'cluster.TF.markers.combo': 'None',
   'cluster.markers.combo _within subclass_': 'None',
   'cluster.markers.combo': 'None'}},
 {'labelset': 'cluster',
  'cell_label': '5197 CB Granule Glut_1',
  'cell_set_accession': 'CS20230722_CLUS_5197',
  'parent_cell_set_accession': 'CS20230722_SUPT_1154',
  'author_annotation_fields': {'neighborhood': 'NN-IMN-GC',
   'subclass.tf.markers.combo': 'None',
   'subclass.markers.combo': 'None',
   'supertype.markers.combo _within subclass_': 'None',
   'supertype.markers.combo':

## Merge

In [19]:
!cas merge --json split_cas.json --anndata CS20230722_CLAS_29.h5ad --validate --output merged_cas_CS20230722_CLAS_29.h5ad

In [20]:
merged_df = ad.read_h5ad("merged_cas_CS20230722_CLAS_29.h5ad", backed="r")

In [21]:
cas_json_str = merged_df.uns["cas"]
cas_json_obj = json.loads(cas_json_str)

In [22]:
print(json.dumps(cas_json_obj, indent=2))

{
  "author_name": "Hongkui Zeng",
  "title": "Whole Mouse Brain Taxonomy",
  "description": "Atlas of whole mouse brain.",
  "labelsets": [
    {
      "name": "class",
      "description": "The top level of cell type definition in the mouse whole brain taxonomy. It is primarily determined by broad brain region and neurotransmitter type. All cells within a subclass belong to the same class. Class provides a broader categorization of cell types.",
      "rank": 3
    },
    {
      "name": "subclass",
      "description": "The coarse level of cell type definition in the mouse whole brain taxonomy. All cells within a supertype belong to the same subclass. Subclass groups together related supertypes.",
      "rank": 2
    },
    {
      "name": "supertype",
      "description": "The second finest level of cell type definition in the mouse whole brain taxonomy. All cells within a cluster belong to the same supertype. Supertype groups together similar clusters.",
      "rank": 1
    },
   

In [23]:
# Convert the annotations to a DataFrame
annotations_df = pd.DataFrame(cas_json_obj["annotations"])

# Display the DataFrame
annotations_df

,labelset,cell_label,cell_set_accession,cell_ontology_term_id,cell_ontology_term,author_annotation_fields,parent_cell_set_accession
0,class,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",NaN
1,cluster,5197 CB Granule Glut_1,CS20230722_CLUS_5197,NaN,NaN,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",CS20230722_SUPT_1154
2,cluster,5198 CB Granule Glut_1,CS20230722_CLUS_5198,NaN,NaN,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",CS20230722_SUPT_1154
3,cluster,5199 CB Granule Glut_1,CS20230722_CLUS_5199,NaN,NaN,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",CS20230722_SUPT_1154
4,cluster,5200 CB Granule Glut_2,CS20230722_CLUS_5200,NaN,NaN,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",CS20230722_SUPT_1155
5,cluster,5201 CB Granule Glut_2,CS20230722_CLUS_5201,NaN,NaN,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",CS20230722_SUPT_1155
6,cluster,5202 DCO UBC Glut_1,CS20230722_CLUS_5202,NaN,NaN,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",CS20230722_SUPT_1156
7,cluster,5203 DCO UBC Glut_1,CS20230722_CLUS_5203,NaN,NaN,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",CS20230722_SUPT_1156
8,cluster,5204 DCO UBC Glut_1,CS20230722_CLUS_5204,NaN,NaN,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",CS20230722_SUPT_1156
9,cluster,5205 DCO UBC Glut_1,CS20230722_CLUS_5205,NaN,NaN,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",CS20230722_SUPT_1156


In [24]:
merged_df.file.close()

## Flatten

In [25]:
!cas flatten --json split_cas.json --anndata merged_cas_CS20230722_CLAS_29.h5ad --output flatten_cas_CS20230722_CLAS_29.h5ad

In [26]:
flatten_df = ad.read_h5ad("flatten_cas_CS20230722_CLAS_29.h5ad", backed="r")

In [27]:
flatten_df.obs

,cell_barcode,library_label,tissue,tissue_ontology_term_id,class,class--cell_set_accession,class--cell_ontology_term_id,class--cell_ontology_term,class--author_annotation_fields,cluster,...,subclass,subclass--cell_set_accession,subclass--cell_ontology_term_id,subclass--cell_ontology_term,subclass--parent_cell_set_accession,subclass--author_annotation_fields,supertype,supertype--cell_set_accession,supertype--parent_cell_set_accession,supertype--author_annotation_fields
cell_label,,,,,,,,,,,,,,,,,,,,,
AAACCCAAGAACAAGG-472_A05,AAACCCAAGAACAAGG,L8TX_201217_01_G07,Cerebellum,UBERON:0002037,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5201 CB Granule Glut_2,...,314 CB Granule Glut,CS20230722_SUBC_314,CL:0001031,cerebellar granule cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",1155 CB Granule Glut_2,CS20230722_SUPT_1155,CS20230722_SUBC_314,"{'neighborhood': 'None', 'subclass.tf.markers...."
AAACCCAAGAATCCCT-473_A06,AAACCCAAGAATCCCT,L8TX_201217_01_A08,Cerebellum,UBERON:0002037,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5201 CB Granule Glut_2,...,314 CB Granule Glut,CS20230722_SUBC_314,CL:0001031,cerebellar granule cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",1155 CB Granule Glut_2,CS20230722_SUPT_1155,CS20230722_SUBC_314,"{'neighborhood': 'None', 'subclass.tf.markers...."
AAACCCAAGACTACCT-225_A01,AAACCCAAGACTACCT,L8TX_200227_01_F10,Medulla,UBERON:0001896,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5197 CB Granule Glut_1,...,314 CB Granule Glut,CS20230722_SUBC_314,CL:0001031,cerebellar granule cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",1154 CB Granule Glut_1,CS20230722_SUPT_1154,CS20230722_SUBC_314,"{'neighborhood': 'None', 'subclass.tf.markers...."
AAACCCAAGAGCTGAC-231.2_B01,AAACCCAAGAGCTGAC,L8TX_200306_01_H12,Medulla,UBERON:0001896,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5197 CB Granule Glut_1,...,314 CB Granule Glut,CS20230722_SUBC_314,CL:0001031,cerebellar granule cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",1154 CB Granule Glut_1,CS20230722_SUPT_1154,CS20230722_SUBC_314,"{'neighborhood': 'None', 'subclass.tf.markers...."
AAACCCAAGAGGACTC-478_A02,AAACCCAAGAGGACTC,L8TX_210107_02_H11,Cerebellum,UBERON:0002037,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5201 CB Granule Glut_2,...,314 CB Granule Glut,CS20230722_SUBC_314,CL:0001031,cerebellar granule cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",1155 CB Granule Glut_2,CS20230722_SUPT_1155,CS20230722_SUBC_314,"{'neighborhood': 'None', 'subclass.tf.markers...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGTCTGATTCT-171_A01,TTTGTTGTCTGATTCT,L8TX_191029_01_C08,Cerebellum,UBERON:0002037,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5201 CB Granule Glut_2,...,314 CB Granule Glut,CS20230722_SUBC_314,CL:0001031,cerebellar granule cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",1155 CB Granule Glut_2,CS20230722_SUPT_1155,CS20230722_SUBC_314,"{'neighborhood': 'None', 'subclass.tf.markers...."
TTTGTTGTCTTCCACG-471_A04,TTTGTTGTCTTCCACG,L8TX_201217_01_A07,Cerebellum,UBERON:0002037,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5201 CB Granule Glut_2,...,314 CB Granule Glut,CS20230722_SUBC_314,CL:0001031,cerebellar granule cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",1155 CB Granule Glut_2,CS20230722_SUPT_1155,CS20230722_SUBC_314,"{'neighborhood': 'None', 'subclass.tf.markers...."
TTTGTTGTCTTGATTC-471_A04,TTTGTTGTCTTGATTC,L8TX_201217_01_A07,Cerebellum,UBERON:

## Edit CAS

In [28]:
flatten_df.obs[flatten_df.obs["supertype"]=="1155 CB Granule Glut_2"]

,cell_barcode,library_label,tissue,tissue_ontology_term_id,class,class--cell_set_accession,class--cell_ontology_term_id,class--cell_ontology_term,class--author_annotation_fields,cluster,...,subclass,subclass--cell_set_accession,subclass--cell_ontology_term_id,subclass--cell_ontology_term,subclass--parent_cell_set_accession,subclass--author_annotation_fields,supertype,supertype--cell_set_accession,supertype--parent_cell_set_accession,supertype--author_annotation_fields
cell_label,,,,,,,,,,,,,,,,,,,,,
AAACCCAAGAACAAGG-472_A05,AAACCCAAGAACAAGG,L8TX_201217_01_G07,Cerebellum,UBERON:0002037,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5201 CB Granule Glut_2,...,314 CB Granule Glut,CS20230722_SUBC_314,CL:0001031,cerebellar granule cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",1155 CB Granule Glut_2,CS20230722_SUPT_1155,CS20230722_SUBC_314,"{'neighborhood': 'None', 'subclass.tf.markers...."
AAACCCAAGAATCCCT-473_A06,AAACCCAAGAATCCCT,L8TX_201217_01_A08,Cerebellum,UBERON:0002037,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5201 CB Granule Glut_2,...,314 CB Granule Glut,CS20230722_SUBC_314,CL:0001031,cerebellar granule cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",1155 CB Granule Glut_2,CS20230722_SUPT_1155,CS20230722_SUBC_314,"{'neighborhood': 'None', 'subclass.tf.markers...."
AAACCCAAGAGGACTC-478_A02,AAACCCAAGAGGACTC,L8TX_210107_02_H11,Cerebellum,UBERON:0002037,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5201 CB Granule Glut_2,...,314 CB Granule Glut,CS20230722_SUBC_314,CL:0001031,cerebellar granule cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",1155 CB Granule Glut_2,CS20230722_SUPT_1155,CS20230722_SUBC_314,"{'neighborhood': 'None', 'subclass.tf.markers...."
AAACCCAAGATGCTGG-471_B04,AAACCCAAGATGCTGG,L8TX_201217_01_B07,Cerebellum,UBERON:0002037,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5201 CB Granule Glut_2,...,314 CB Granule Glut,CS20230722_SUBC_314,CL:0001031,cerebellar granule cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",1155 CB Granule Glut_2,CS20230722_SUPT_1155,CS20230722_SUBC_314,"{'neighborhood': 'None', 'subclass.tf.markers...."
AAACCCAAGCAGGGAG-213.2_B01,AAACCCAAGCAGGGAG,L8TX_200206_01_A03,Cerebellum,UBERON:0002037,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5201 CB Granule Glut_2,...,314 CB Granule Glut,CS20230722_SUBC_314,CL:0001031,cerebellar granule cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",1155 CB Granule Glut_2,CS20230722_SUPT_1155,CS20230722_SUBC_314,"{'neighborhood': 'None', 'subclass.tf.markers...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGTCTGATTCT-171_A01,TTTGTTGTCTGATTCT,L8TX_191029_01_C08,Cerebellum,UBERON:0002037,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5201 CB Granule Glut_2,...,314 CB Granule Glut,CS20230722_SUBC_314,CL:0001031,cerebellar granule cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",1155 CB Granule Glut_2,CS20230722_SUPT_1155,CS20230722_SUBC_314,"{'neighborhood': 'None', 'subclass.tf.markers...."
TTTGTTGTCTTCCACG-471_A04,TTTGTTGTCTTCCACG,L8TX_201217_01_A07,Cerebellum,UBERON:0002037,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5201 CB Granule Glut_2,...,314 CB Granule Glut,CS20230722_SUBC_314,CL:0001031,cerebellar granule cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",1155 CB Granule Glut_2,CS20230722_SUPT_1155,CS20230722_SUBC_314,"{'neighborhood': 'None', 'subclass.tf.markers...."
TTTGTTGTCTTGATTC-471_A04,TTTGTTGTCTTGATTC,L8TX_201217_01_A07,Cerebellum,U

Moving `supertype`-`1155 CB Granule Glut_2` from `subclass`-`314 CB Granule Glut` to `subclass`-`315 DCO UBC Glut`

In [29]:
CB_Granule_Glut = [annotation for annotation in split_cas_w_cids["annotations"] if annotation["cell_label"] == "314 CB Granule Glut"][0]
DCO_UBC_Glut = [annotation for annotation in split_cas_w_cids["annotations"] if annotation["cell_label"] == "315 DCO UBC Glut"][0]
CB_Granule_Glut_2 = [annotation for annotation in split_cas_w_cids["annotations"] if annotation["cell_label"] == "1155 CB Granule Glut_2"][0]

In [30]:
CB_Granule_Glut

{'labelset': 'subclass',
 'cell_label': '314 CB Granule Glut',
 'cell_set_accession': 'CS20230722_SUBC_314',
 'cell_ontology_term_id': 'CL:0001031',
 'cell_ontology_term': 'cerebellar granule cell',
 'parent_cell_set_accession': 'CS20230722_CLAS_29',
 'author_annotation_fields': {'neighborhood': 'NN-IMN-GC',
  'subclass.tf.markers.combo': 'Pax6,Neurod2,Etv1',
  'subclass.markers.combo': 'Gabra6,Ror1',
  'supertype.markers.combo _within subclass_': 'None',
  'supertype.markers.combo': 'None',
  'anatomical_annotation': 'None',
  'merfish.markers.combo': 'None',
  'cluster.TF.markers.combo': 'None',
  'cluster.markers.combo _within subclass_': 'None',
  'cluster.markers.combo': 'None'}}

In [31]:
# len(CB_Granule_Glut["cell_ids"])

In [32]:
# CB_Granule_Glut["cell_ids"] = list(set(CB_Granule_Glut["cell_ids"]) - set(CB_Granule_Glut_2["cell_ids"]))

In [33]:
# len(CB_Granule_Glut["cell_ids"])

In [34]:
DCO_UBC_Glut

{'labelset': 'subclass',
 'cell_label': '315 DCO UBC Glut',
 'cell_set_accession': 'CS20230722_SUBC_315',
 'cell_ontology_term_id': 'CL:4023161',
 'cell_ontology_term': 'unipolar brush cell',
 'parent_cell_set_accession': 'CS20230722_CLAS_29',
 'author_annotation_fields': {'neighborhood': 'NN-IMN-GC',
  'subclass.tf.markers.combo': 'Eomes,Lmx1a,Klf3',
  'subclass.markers.combo': 'Sln,Lmx1a',
  'supertype.markers.combo _within subclass_': 'None',
  'supertype.markers.combo': 'None',
  'anatomical_annotation': 'None',
  'merfish.markers.combo': 'None',
  'cluster.TF.markers.combo': 'None',
  'cluster.markers.combo _within subclass_': 'None',
  'cluster.markers.combo': 'None'}}

In [35]:
# len(DCO_UBC_Glut["cell_ids"])

In [36]:
# DCO_UBC_Glut["cell_ids"] = DCO_UBC_Glut["cell_ids"] + CB_Granule_Glut_2["cell_ids"]

In [37]:
# len(DCO_UBC_Glut["cell_ids"])

In [38]:
CB_Granule_Glut_2

{'labelset': 'supertype',
 'cell_label': '1155 CB Granule Glut_2',
 'cell_set_accession': 'CS20230722_SUPT_1155',
 'parent_cell_set_accession': 'CS20230722_SUBC_314',
 'author_annotation_fields': {'neighborhood': 'None',
  'subclass.tf.markers.combo': 'None',
  'subclass.markers.combo': 'None',
  'supertype.markers.combo _within subclass_': 'Gap43',
  'supertype.markers.combo': 'Gabra6,Gap43,Rab37',
  'anatomical_annotation': 'None',
  'merfish.markers.combo': 'None',
  'cluster.TF.markers.combo': 'None',
  'cluster.markers.combo _within subclass_': 'None',
  'cluster.markers.combo': 'None'}}

In [39]:
CB_Granule_Glut_2["cell_label"] = "Updated_CB Granule Glut_2"
CB_Granule_Glut_2["parent_cell_set_accession"] = DCO_UBC_Glut["cell_set_accession"]

In [40]:
with open("edited_split_cas.json", "w") as f:
    f.write(json.dumps(split_cas_w_cids, indent=2))

In [41]:
!cas flatten --json edited_split_cas.json --anndata merged_cas_CS20230722_CLAS_29.h5ad --output edited_flatten_cas_CS20230722_CLAS_29.h5ad

In [42]:
edited_flatten_df = ad.read_h5ad("edited_flatten_cas_CS20230722_CLAS_29.h5ad")

In [43]:
edited_flatten_df.obs[edited_flatten_df.obs["supertype"]=="Updated_CB Granule Glut_2"]

,cell_barcode,library_label,tissue,tissue_ontology_term_id,class,class--cell_set_accession,class--cell_ontology_term_id,class--cell_ontology_term,class--author_annotation_fields,cluster,...,subclass,subclass--cell_set_accession,subclass--cell_ontology_term_id,subclass--cell_ontology_term,subclass--parent_cell_set_accession,subclass--author_annotation_fields,supertype,supertype--cell_set_accession,supertype--parent_cell_set_accession,supertype--author_annotation_fields
cell_label,,,,,,,,,,,,,,,,,,,,,
AAACCCAAGAACAAGG-472_A05,AAACCCAAGAACAAGG,L8TX_201217_01_G07,Cerebellum,UBERON:0002037,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5201 CB Granule Glut_2,...,315 DCO UBC Glut,CS20230722_SUBC_315,CL:4023161,unipolar brush cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",Updated_CB Granule Glut_2,CS20230722_SUPT_1155,CS20230722_SUBC_315,"{'neighborhood': 'None', 'subclass.tf.markers...."
AAACCCAAGAATCCCT-473_A06,AAACCCAAGAATCCCT,L8TX_201217_01_A08,Cerebellum,UBERON:0002037,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5201 CB Granule Glut_2,...,315 DCO UBC Glut,CS20230722_SUBC_315,CL:4023161,unipolar brush cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",Updated_CB Granule Glut_2,CS20230722_SUPT_1155,CS20230722_SUBC_315,"{'neighborhood': 'None', 'subclass.tf.markers...."
AAACCCAAGAGGACTC-478_A02,AAACCCAAGAGGACTC,L8TX_210107_02_H11,Cerebellum,UBERON:0002037,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5201 CB Granule Glut_2,...,315 DCO UBC Glut,CS20230722_SUBC_315,CL:4023161,unipolar brush cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",Updated_CB Granule Glut_2,CS20230722_SUPT_1155,CS20230722_SUBC_315,"{'neighborhood': 'None', 'subclass.tf.markers...."
AAACCCAAGATGCTGG-471_B04,AAACCCAAGATGCTGG,L8TX_201217_01_B07,Cerebellum,UBERON:0002037,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5201 CB Granule Glut_2,...,315 DCO UBC Glut,CS20230722_SUBC_315,CL:4023161,unipolar brush cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",Updated_CB Granule Glut_2,CS20230722_SUPT_1155,CS20230722_SUBC_315,"{'neighborhood': 'None', 'subclass.tf.markers...."
AAACCCAAGCAGGGAG-213.2_B01,AAACCCAAGCAGGGAG,L8TX_200206_01_A03,Cerebellum,UBERON:0002037,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5201 CB Granule Glut_2,...,315 DCO UBC Glut,CS20230722_SUBC_315,CL:4023161,unipolar brush cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",Updated_CB Granule Glut_2,CS20230722_SUPT_1155,CS20230722_SUBC_315,"{'neighborhood': 'None', 'subclass.tf.markers...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGTCTGATTCT-171_A01,TTTGTTGTCTGATTCT,L8TX_191029_01_C08,Cerebellum,UBERON:0002037,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5201 CB Granule Glut_2,...,315 DCO UBC Glut,CS20230722_SUBC_315,CL:4023161,unipolar brush cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",Updated_CB Granule Glut_2,CS20230722_SUPT_1155,CS20230722_SUBC_315,"{'neighborhood': 'None', 'subclass.tf.markers...."
TTTGTTGTCTTCCACG-471_A04,TTTGTTGTCTTCCACG,L8TX_201217_01_A07,Cerebellum,UBERON:0002037,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5201 CB Granule Glut_2,...,315 DCO UBC Glut,CS20230722_SUBC_315,CL:4023161,unipolar brush cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",Updated_CB Granule Glut_2,CS20230722_SUPT_1155,CS20230722_SUBC_315,"{'neighborhood': 'None', 'subclass.tf.markers...."
TTTGTTGTCTTGATTC-471_A04,TTTGTTGTCTTGATTC,L8TX_201217_01_A07,Cerebellum,UBERON:0002037,29 CB Glut,CS2

## Edit OBS

In [44]:
edited_flatten_df.obs["subclass"] = edited_flatten_df.obs["subclass"].replace("315 DCO UBC Glut", "Upgraded DCO UBC Glut")
edited_flatten_df.obs["subclass"] = edited_flatten_df.obs["subclass"].replace("314 CB Granule Glut", "Downgraded CB Granule Glut")

/var/folders/17/gqg9r9wd5x9_dn6d2sf4hd1h0000gq/T/ipykernel_83811/3767000135.py:1: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  edited_flatten_df.obs["subclass"] = edited_flatten_df.obs["subclass"].replace("315 DCO UBC Glut", "Upgraded DCO UBC Glut")
/var/folders/17/gqg9r9wd5x9_dn6d2sf4hd1h0000gq/T/ipykernel_83811/3767000135.py:2: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  edited_flatten_df.obs["subclass"] = edited_flatten_df.obs["subclass"].replace("314 CB Granule Glut", "Downgraded CB Granule Glut")


In [45]:
edited_flatten_df.obs[edited_flatten_df.obs["supertype"]=="Updated_CB Granule Glut_2"]

,cell_barcode,library_label,tissue,tissue_ontology_term_id,class,class--cell_set_accession,class--cell_ontology_term_id,class--cell_ontology_term,class--author_annotation_fields,cluster,...,subclass,subclass--cell_set_accession,subclass--cell_ontology_term_id,subclass--cell_ontology_term,subclass--parent_cell_set_accession,subclass--author_annotation_fields,supertype,supertype--cell_set_accession,supertype--parent_cell_set_accession,supertype--author_annotation_fields
cell_label,,,,,,,,,,,,,,,,,,,,,
AAACCCAAGAACAAGG-472_A05,AAACCCAAGAACAAGG,L8TX_201217_01_G07,Cerebellum,UBERON:0002037,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5201 CB Granule Glut_2,...,Upgraded DCO UBC Glut,CS20230722_SUBC_315,CL:4023161,unipolar brush cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",Updated_CB Granule Glut_2,CS20230722_SUPT_1155,CS20230722_SUBC_315,"{'neighborhood': 'None', 'subclass.tf.markers...."
AAACCCAAGAATCCCT-473_A06,AAACCCAAGAATCCCT,L8TX_201217_01_A08,Cerebellum,UBERON:0002037,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5201 CB Granule Glut_2,...,Upgraded DCO UBC Glut,CS20230722_SUBC_315,CL:4023161,unipolar brush cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",Updated_CB Granule Glut_2,CS20230722_SUPT_1155,CS20230722_SUBC_315,"{'neighborhood': 'None', 'subclass.tf.markers...."
AAACCCAAGAGGACTC-478_A02,AAACCCAAGAGGACTC,L8TX_210107_02_H11,Cerebellum,UBERON:0002037,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5201 CB Granule Glut_2,...,Upgraded DCO UBC Glut,CS20230722_SUBC_315,CL:4023161,unipolar brush cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",Updated_CB Granule Glut_2,CS20230722_SUPT_1155,CS20230722_SUBC_315,"{'neighborhood': 'None', 'subclass.tf.markers...."
AAACCCAAGATGCTGG-471_B04,AAACCCAAGATGCTGG,L8TX_201217_01_B07,Cerebellum,UBERON:0002037,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5201 CB Granule Glut_2,...,Upgraded DCO UBC Glut,CS20230722_SUBC_315,CL:4023161,unipolar brush cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",Updated_CB Granule Glut_2,CS20230722_SUPT_1155,CS20230722_SUBC_315,"{'neighborhood': 'None', 'subclass.tf.markers...."
AAACCCAAGCAGGGAG-213.2_B01,AAACCCAAGCAGGGAG,L8TX_200206_01_A03,Cerebellum,UBERON:0002037,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5201 CB Granule Glut_2,...,Upgraded DCO UBC Glut,CS20230722_SUBC_315,CL:4023161,unipolar brush cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",Updated_CB Granule Glut_2,CS20230722_SUPT_1155,CS20230722_SUBC_315,"{'neighborhood': 'None', 'subclass.tf.markers...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGTCTGATTCT-171_A01,TTTGTTGTCTGATTCT,L8TX_191029_01_C08,Cerebellum,UBERON:0002037,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5201 CB Granule Glut_2,...,Upgraded DCO UBC Glut,CS20230722_SUBC_315,CL:4023161,unipolar brush cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",Updated_CB Granule Glut_2,CS20230722_SUPT_1155,CS20230722_SUBC_315,"{'neighborhood': 'None', 'subclass.tf.markers...."
TTTGTTGTCTTCCACG-471_A04,TTTGTTGTCTTCCACG,L8TX_201217_01_A07,Cerebellum,UBERON:0002037,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5201 CB Granule Glut_2,...,Upgraded DCO UBC Glut,CS20230722_SUBC_315,CL:4023161,unipolar brush cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",Updated_CB Granule Glut_2,CS20230722_SUPT_1155,CS20230722_SUBC_315,"{'neighborhood': 'None', 'subclass.tf.markers...."
TTTGTTGTCTTGATTC-471_A04,TTTGTTGTCTTGATTC,L8TX_201217_01_A07,Cereb

In [46]:
edited_flatten_df.obs[edited_flatten_df.obs["subclass"]=="Downgraded CB Granule Glut"]

,cell_barcode,library_label,tissue,tissue_ontology_term_id,class,class--cell_set_accession,class--cell_ontology_term_id,class--cell_ontology_term,class--author_annotation_fields,cluster,...,subclass,subclass--cell_set_accession,subclass--cell_ontology_term_id,subclass--cell_ontology_term,subclass--parent_cell_set_accession,subclass--author_annotation_fields,supertype,supertype--cell_set_accession,supertype--parent_cell_set_accession,supertype--author_annotation_fields
cell_label,,,,,,,,,,,,,,,,,,,,,
AAACCCAAGACTACCT-225_A01,AAACCCAAGACTACCT,L8TX_200227_01_F10,Medulla,UBERON:0001896,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5197 CB Granule Glut_1,...,Downgraded CB Granule Glut,CS20230722_SUBC_314,CL:0001031,cerebellar granule cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",1154 CB Granule Glut_1,CS20230722_SUPT_1154,CS20230722_SUBC_314,"{'neighborhood': 'None', 'subclass.tf.markers...."
AAACCCAAGAGCTGAC-231.2_B01,AAACCCAAGAGCTGAC,L8TX_200306_01_H12,Medulla,UBERON:0001896,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5197 CB Granule Glut_1,...,Downgraded CB Granule Glut,CS20230722_SUBC_314,CL:0001031,cerebellar granule cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",1154 CB Granule Glut_1,CS20230722_SUPT_1154,CS20230722_SUBC_314,"{'neighborhood': 'None', 'subclass.tf.markers...."
AAACCCAAGCCGATCC-175_A01,AAACCCAAGCCGATCC,L8TX_191105_01_F09,Medulla,UBERON:0001896,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5197 CB Granule Glut_1,...,Downgraded CB Granule Glut,CS20230722_SUBC_314,CL:0001031,cerebellar granule cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",1154 CB Granule Glut_1,CS20230722_SUPT_1154,CS20230722_SUBC_314,"{'neighborhood': 'None', 'subclass.tf.markers...."
AAACCCAAGGCATGGT-179_A01,AAACCCAAGGCATGGT,L8TX_191119_01_B11,Medulla,UBERON:0001896,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5197 CB Granule Glut_1,...,Downgraded CB Granule Glut,CS20230722_SUBC_314,CL:0001031,cerebellar granule cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",1154 CB Granule Glut_1,CS20230722_SUPT_1154,CS20230722_SUBC_314,"{'neighborhood': 'None', 'subclass.tf.markers...."
AAACCCAAGGTCCAGA-225_B01,AAACCCAAGGTCCAGA,L8TX_200227_01_C10,Medulla,UBERON:0001896,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5197 CB Granule Glut_1,...,Downgraded CB Granule Glut,CS20230722_SUBC_314,CL:0001031,cerebellar granule cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",1154 CB Granule Glut_1,CS20230722_SUPT_1154,CS20230722_SUBC_314,"{'neighborhood': 'None', 'subclass.tf.markers...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGGTTACAGCT-175_A01,TTTGTTGGTTACAGCT,L8TX_191105_01_F09,Medulla,UBERON:0001896,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5197 CB Granule Glut_1,...,Downgraded CB Granule Glut,CS20230722_SUBC_314,CL:0001031,cerebellar granule cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",1154 CB Granule Glut_1,CS20230722_SUPT_1154,CS20230722_SUBC_314,"{'neighborhood': 'None', 'subclass.tf.markers...."
TTTGTTGGTTGTACGT-175_A01,TTTGTTGGTTGTACGT,L8TX_191105_01_F09,Medulla,UBERON:0001896,29 CB Glut,CS20230722_CLAS_29,CL:0000540,neuron,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",5198 CB Granule Glut_1,...,Downgraded CB Granule Glut,CS20230722_SUBC_314,CL:0001031,cerebellar granule cell,CS20230722_CLAS_29,"{'neighborhood': 'NN-IMN-GC', 'subclass.tf.mar...",1154 CB Granule Glut_1,CS20230722_SUPT_1154,CS20230722_SUBC_314,"{'neighborhood': 'None', 'subclass.tf.markers...."
TTTGTTGTCCTTATAC-179_B01,TTTGTTGTCCTTATAC,L8T

In [47]:
edited_flatten_df.write("new_edited_flatten_cas_CS20230722_CLAS_29.h5ad", compression="gzip")

In [48]:
edited_flatten_df.file.close()

## Unflatten

In [49]:
!cas unflatten --anndata new_edited_flatten_cas_CS20230722_CLAS_29.h5ad --json edited_split_cas.json --output_anndata unflatten_cas_CS20230722_CLAS_29.h5ad  --output_json new_split_cas.json

In [50]:
with open("new_split_cas.json", "r") as f:
    new_split_cas = json.load(f)

In [51]:
[anno for anno in new_split_cas["annotations"] if anno["labelset"] == "subclass"]

[{'author_annotation_fields': {'neighborhood': 'NN-IMN-GC',
   'subclass.tf.markers.combo': 'Pax6,Neurod2,Etv1',
   'subclass.markers.combo': 'Gabra6,Ror1',
   'supertype.markers.combo _within subclass_': 'None',
   'supertype.markers.combo': 'None',
   'anatomical_annotation': 'None',
   'merfish.markers.combo': 'None',
   'cluster.TF.markers.combo': 'None',
   'cluster.markers.combo _within subclass_': 'None',
   'cluster.markers.combo': 'None'},
  'labelset': 'subclass',
  'cell_label': 'Downgraded CB Granule Glut',
  'cell_set_accession': 'CS20230722_SUBC_314',
  'cell_ontology_term_id': 'CL:0001031',
  'cell_ontology_term': 'cerebellar granule cell',
  'parent_cell_set_accession': 'CS20230722_CLAS_29'},
 {'author_annotation_fields': {'neighborhood': 'NN-IMN-GC',
   'subclass.tf.markers.combo': 'Eomes,Lmx1a,Klf3',
   'subclass.markers.combo': 'Sln,Lmx1a',
   'supertype.markers.combo _within subclass_': 'None',
   'supertype.markers.combo': 'None',
   'anatomical_annotation': 'None'